# Sprawozdanie 2

Sprawozdanie zostało opracowane w oparciu o Python 3.7 i wersję opencv-contrib 3.4.2.17 tak, aby uniknąć konieczności ręcznej kompilacji biblioteki, by wykorzystać algorytm SURF (SIFT jest otwarcie dostępny od 2020). Dla nowszych wersji Pythona i biblioteki OpenCV mogą być konieczne lekkie zmiany w kodzie.

Lista wymaganych plików w tym samym folderze co notebook:
- lab5_1.jpg
- lab5_2.png
- vid1.mov
- vid2.mov
- haarcascade_frontalface_default.xml
- face_example.png
- vecteezy_portrait-of-young-beautiful-woman-with-smooth-healthy-skin__119.mov (wskazane jest miejsce gdzie należy podać własne wideo z twarzą)

## Znajdowanie krawędzi, transformacja Hough dla linii i okręgów

Przykładowy obrazek.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import cv2

image = cv2.imread('lab5_1.jpg')

gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

plt.imshow(gray_image, cmap='gray')

### Porównanie metod Laplace i metody Canny do znajdowania krawędzi oraz wpływ szumu gaussowskiego na ich działanie

In [ ]:
gray_image_blurred = cv2.GaussianBlur(gray_image, (5, 5), 0)

laplace_edge = cv2.Laplacian(gray_image, cv2.CV_16UC1, ksize=5)
laplace_edge_blurred = cv2.Laplacian(gray_image_blurred, cv2.CV_16UC1, ksize=5)

plt.subplot(3, 1, 1)
plt.title("Oryginał")
plt.imshow(gray_image, cmap='gray')

plt.subplot(3, 1, 2)
plt.title("Laplace - brak dodanego szumu w obrazie")
plt.imshow(laplace_edge, cmap='gray')

plt.subplot(3, 1, 3)
plt.title("Laplace - dodany szum")
plt.imshow(laplace_edge_blurred, cmap='gray')

plt.gcf().set_size_inches((30, 20))

In [ ]:
canny_edge = cv2.Canny(gray_image, 100, 256)
canny_edge_blurred = cv2.Canny(gray_image_blurred, 100, 256)

plt.subplot(3, 1, 1)
plt.title("Oryginał")
plt.imshow(gray_image, cmap='gray')

plt.subplot(3, 1, 2)
plt.title("Laplace - brak dodanego szumu w obrazie")
plt.imshow(canny_edge, cmap='gray')

plt.subplot(3, 1, 3)
plt.title("Laplace - dodany szum")
plt.imshow(canny_edge_blurred, cmap='gray')

plt.gcf().set_size_inches((30, 20))

Metoda Laplace jest bardzo czuła na szum w obrazie, co wynika z jej sposobu działania. Jako, że metoda ta próbuje obliczyć druga pochodną w obszarze obrazu, a ta z kolei jest bardzo czuła na najmniejsze zmiany. W porównaniu metoda Canny produkuje lepszy wynik, ponieważ nie stosuje pochodnych drugiego rzędu oraz ma metody "poprawiania" krawędzi.

W obydwu przypadkach dodanie szumu przed uruchomieniem metod powoduje usunięcie niektórych krawędzi w porównaniu z poprzednimi rezultatami, a przypadku metody Laplace niektóre krawędzie stają również pogrubione i rozmazane.

### Wpływ parametrów minVal, maxVal i kSize na działanie algorytmu Canny.

In [ ]:
minVals = [100, 200, 230]
maxVals = [128, 200, 256]
kSizes = [3, 5, 7]

for i in range(len(minVals)):
    canny_edge = cv2.Canny(gray_image, threshold1=minVals[i], threshold2=256)

    plt.subplot(len(minVals), 1, i + 1)
    plt.title(f'Canny minVal = {minVals[i]}')
    plt.imshow(canny_edge, cmap='gray')

plt.gcf().set_size_inches((30, 20))

In [ ]:
for i in range(len(maxVals)):
    canny_edge = cv2.Canny(gray_image, threshold1=100, threshold2=maxVals[i])

    plt.subplot(len(maxVals), 1, i + 1)
    plt.title(f'Canny maxVal = {maxVals[i]}')
    plt.imshow(canny_edge, cmap='gray')

plt.gcf().set_size_inches((30, 20))

In [ ]:
for i in range(len(kSizes)):
    canny_edge = cv2.Canny(gray_image, threshold1=100, threshold2=256, apertureSize=kSizes[i])

    plt.subplot(len(kSizes), 1, i + 1)
    plt.title(f'Canny kSize = {kSizes[i]}')
    plt.imshow(canny_edge, cmap='gray')

plt.gcf().set_size_inches((30, 20))

Wpływy parametrów:
- minVal - wraz ze wzrostem wartości krótkie krawędzie nie są przedłużane
- maxVal - wraz ze wzrostem wartości krótkie krawędzie są usuwane
- kSize - wraz ze wzrostem pojawia się wiele szumu w obrazie wynikowym

### Wykrywanie linii metodą transformacji Hough.

In [ ]:
img = gray_image.copy()
gray_image_blurred = cv2.GaussianBlur(img, (5, 5), 0)
canny_blurred_edges = cv2.Canny(gray_image_blurred,50,150,apertureSize = 3)
lines = cv2.HoughLines(canny_blurred_edges,1,np.pi/180,200)

for line in lines:
    rho, theta = line[0]
    a = np.cos(theta)
    b = np.sin(theta)
    x0 = a*rho
    y0 = b*rho
    x1 = int(x0 + 1000*(-b))
    y1 = int(y0 + 1000*(a))
    x2 = int(x0 - 1000*(-b))
    y2 = int(y0 - 1000*(a))
    cv2.line(img,(x1,y1),(x2,y2),(0,0,255),2)

plt.imshow(img, cmap='gray')
plt.gcf().set_size_inches((20, 10))

### Wpływ progu na działanie transformacji Hough.

In [ ]:
hough_threshs = [100, 150, 200]

for i in range(len(hough_threshs)):
    hough_thresh = hough_threshs[i]
    img = gray_image.copy()
    gray_image_blurred = cv2.GaussianBlur(gray_image, (5, 5), 0)
    canny_blurred_edges = cv2.Canny(gray_image_blurred,50,150,apertureSize = 3)
    lines = cv2.HoughLines(canny_blurred_edges,1,np.pi/180,hough_thresh)

    for line in lines:
        rho, theta = line[0]
        a = np.cos(theta)
        b = np.sin(theta)
        x0 = a*rho
        y0 = b*rho
        x1 = int(x0 + 1000*(-b))
        y1 = int(y0 + 1000*(a))
        x2 = int(x0 - 1000*(-b))
        y2 = int(y0 - 1000*(a))
        cv2.line(img,(x1,y1),(x2,y2),(0,0,255),2)

    plt.subplot(len(hough_threshs), 1, i + 1)
    plt.title(f'Hough z progiem = {hough_thresh}')
    plt.imshow(img, cmap='gray')

plt.gcf().set_size_inches((30, 20))

Przewidywalnie próg w transformacji Hough wraz ze swoim wzrostem powoduje zmniejszenie ilości wykrytych linii w obrazie, jako że każda linia potrzebuje więcej "głosów", by zostać przyjęta.

### Wykrywanie okręgów przy pomocy transformacji Hough.

Przykładowy obrazek.

In [ ]:
img = cv2.imread('lab5_2.png')
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

plt.imshow(img)

In [ ]:
gimg = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
gimg = cv2.GaussianBlur(gimg, (5, 5), 0)

circles = cv2.HoughCircles(gimg,cv2.HOUGH_GRADIENT,1,100,param1=50,param2=30,minRadius=150,maxRadius=200)
circles = np.uint16(np.around(circles))

for circle in circles[0]:
    cv2.circle(img, (circle[0], circle[1]), circle[2], (255, 0, 0), 3)

plt.imshow(img)
plt.gcf().set_size_inches((20, 10))

Przy relatywnie prostej modyfikacji oryginalnej metody do wykrywania linii Hough można również wykrywać okręgi na obrazie. W tej modyfikacji kształty, które są kandydatami są opisane nieco inaczej niż przypadku linii (kąt i promień od początku układy współrzędnych), ale działanie jest mimo wszystko bardzo podobne.

## Transformacja Hough i wykrywanie ruchu w filmie wideo

### Wykrywanie linii w przykładowym filmie przy pomocy transformacji Hough.

In [ ]:
def process_video(filename, image_fun, outfile):
    cap = cv2.VideoCapture(filename)

    if cap.isOpened():

        vwidth  = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
        vheight = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
        vfps = cap.get(cv2.CAP_PROP_FPS)
        print(vwidth, vheight, vfps)

        fourcc = cv2.VideoWriter_fourcc(*'XVID')
        out = cv2.VideoWriter(outfile, fourcc, vfps, (vwidth, vheight))

        while True:
            ret, frame = cap.read()

            if not ret:
                break

            gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

            processed_frame = image_fun(gray_frame)

            processed_frame = cv2.cvtColor(processed_frame, cv2.COLOR_GRAY2BGR)

            out.write(processed_frame)

        out.release()
        cap.release()

In [ ]:
def hough_lines(gray_image, kernelsize=3):
    img = gray_image.copy()
    gray_image_blurred = cv2.medianBlur(gray_image, kernelsize)
    canny_blurred_edges = cv2.Canny(gray_image_blurred,50,150,apertureSize = kernelsize)
    lines = cv2.HoughLines(canny_blurred_edges,1,np.pi/180,200)

    for line in lines:
        rho, theta = line[0]
        a = np.cos(theta)
        b = np.sin(theta)
        x0 = a*rho
        y0 = b*rho
        x1 = int(x0 + 10000*(-b))
        y1 = int(y0 + 10000*(a))
        x2 = int(x0 - 10000*(-b))
        y2 = int(y0 - 10000*(a))
        cv2.line(img,(x1,y1),(x2,y2),(0,0,255),2)

    return img

In [ ]:
process_video('vid1.mov', hough_lines, 'vid1_processed.avi')

Plik wejściowy do wykrywania linii - "vid1.mov".

Plik wyjściowy z narysowanymi wykrytymi liniami - "vid1_processed.avi".

### Wykrywanie poruszających się obiektów przy pomocy prostej różnicy.

In [ ]:
def process_video2(filename, outfile, threshold):
    cap = cv2.VideoCapture(filename)

    if cap.isOpened():

        vwidth  = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
        vheight = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
        vfps = cap.get(cv2.CAP_PROP_FPS)
        print(vwidth, vheight, vfps)

        fourcc = cv2.VideoWriter_fourcc(*'XVID')
        out = cv2.VideoWriter(outfile, fourcc, vfps, (vwidth, vheight))

        first_frame = None

        while True:
            ret, frame = cap.read()

            if not ret:
                break

            if first_frame is None:
                first_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
                continue

            second_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

            diff_frame = cv2.subtract(first_frame, second_frame)
            _, diff_frame = cv2.threshold(diff_frame, threshold, 255, cv2.THRESH_BINARY)

            processed_frame = cv2.bitwise_and(frame, frame, mask=diff_frame)

            out.write(processed_frame)

            first_frame = second_frame

        out.release()
        cap.release()

In [ ]:
process_video2('vid2.mov', 'vid2_processed.avi', 40)

Plik wejściowy do wykrywania poruszających się obiektów - "vid2.mov".

Plik wyjściowy z poruszającymi się obiektami - "vid2_processed.avi".

Plik wynikowy pokazuje jak w prosty sposób można próbować wykrywać poruszające się na obrazie obiekty. Metoda jest jednak podatna na małe zmiany obrazie, więc nie jest idealna.

### Wyszukiwanie elementów interesujących różnymi metodami w obrazie, wykrywanie twarzy w filmie wideo przy pomocy klasyfikatora kaskadowego

Przykładowy obrazek.

In [ ]:
image = cv2.imread('lab5_2.png')
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
gray_image = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)

plt.imshow(image)

### Wykrywanie rogów metodą Harrisa.

In [ ]:
import time

harris_thresh = 125000
blockSize = 2
apertureSize = 3
k = 0.04

start = time.time()
dst = cv2.cornerHarris(gray_image.astype(np.float32), blockSize, apertureSize, k)
end = time.time()

print(f'cornerHarris() zajął {end - start} sekund')

out = image.copy()

for i in range(dst.shape[0]):
    for j in range(dst.shape[1]):
        if int(dst[i,j]) > harris_thresh:
            cv2.circle(out, (j,i), 5, (255, 0, 255), 1)

plt.imshow(out)
plt.gcf().set_size_inches((30, 20))

### Wykrywanie rogów metodą Shi-Thomasi.

In [ ]:
start = time.time()
dst = cv2.goodFeaturesToTrack(gray_image, 1500, 0.01, 10)
end = time.time()

print(f'goodFeaturesToTrack() zajął {end - start} sekund')

out = image.copy()

for circle in dst:
    circle = circle[0]
    cv2.circle(out, (circle[0],circle[1]), 5, (255, 0, 255), 1)

plt.imshow(out)
plt.gcf().set_size_inches((30, 20))

### Metoda wykrywania interesujących elementów SIFT.

In [ ]:
# w nowszej wersji OpenCV - cv2.SIFT_create()
sift = cv2.xfeatures2d.SIFT_create(nfeatures=1500)

start = time.time()
kp = sift.detect(gray_image, None)
end = time.time()

print(f'sift.detect() zajął {end - start} sekund')

img = image.copy()

img = cv2.drawKeypoints(img, kp, None, color=(255, 0, 255))

plt.imshow(img)
plt.gcf().set_size_inches((30, 20))

### Metoda wykrywania interesujących elementów SURF.

In [ ]:
surf = cv2.xfeatures2d.SURF_create(4000)

start = time.time()
kp = surf.detect(gray_image, None)
end = time.time()

print(f'surf.detect() zajął {end - start} sekund')

img = image.copy()

img = cv2.drawKeypoints(img, kp, None, color=(255, 0, 255))

plt.imshow(img)
plt.gcf().set_size_inches((30, 20))

### Metoda wykrywania interesujących elementów FAST.

In [ ]:
fast = cv2.FastFeatureDetector_create(threshold=30)

start = time.time()
kp = fast.detect(gray_image, None)
end = time.time()

print(f'fast.detect() zajął {end - start} sekund')

img = image.copy()

img = cv2.drawKeypoints(img, kp, None, color=(255, 0, 255))

plt.imshow(img)
plt.gcf().set_size_inches((30, 20))

### Metoda wykrywania interesujących elementów ORB.

In [ ]:
orb = cv2.ORB_create(nfeatures=1500, fastThreshold=30)

start = time.time()
kp = orb.detect(gray_image, None)
end = time.time()

print(f'orb.detect() zajął {end - start} sekund')

img = image.copy()

img = cv2.drawKeypoints(img, kp, None, color=(255, 0, 255))

plt.imshow(img)
plt.gcf().set_size_inches((30, 20))

### Porównanie metod wykrywania elementów interesujących.

We wszystkich metodach tak dobrano parametry (lub explicite zadano maksymalną ilość znalezionych elementów), by ich ilość wynosiła ~1500.

Metoda:
- metoda Harrisa - działa dość szybko (ale nie dość by działać, np. w czasie rzeczywistym z 30 fps) i pozwala znaleźć dość dużą ilość rogów, ale nie wszystkie; dodatkowo wymaga podania pewnego progu przyjęcia, co może być potencjalnie problematyczne
- metoda Shi-Thomasi - pozwala również znajdować rogi w obrazie, ponieważ od metody Harrisa różni się jedynie sposobem akceptowania punktów, działa wolniej od niej wolniej, ale wydaje się zwracać lepsze wyniki, choć czasem wpada w pułapkę artefaktów w obrazie (np. znak parkingu w okolicach zaokrąglenia litery P)
- metoda SIFT - najwolniejsza z metod, zwraca całkiem dobre wyniki, ale jej mocne strony w tym badaniu się nie okazują, a są one takie, że dla tej metody nie trzeba ręcznie za każdym razem dostrajać parametrów, jak w przypadku metody Harrisa, kiedy pracuje się z obrazem o innym rozmiarze albo kiedy elementy interesujące są większe od okna stosowanego w metodzie Harrisa
- metoda SURF - działa podobnie jak SIFT, ale znacznie szybciej
- metoda FAST - najszybsza z badanych metod, znajdowane przez nią rogi najlepiej pokrywają się z oczekiwaniami, ma natomiast poważne wady - wymaga podania wartości progu oraz nie radzi sobie z rogami, które nie mieszczą się w jej "oknie" (okręgu)
- metoda ORB - druga najszybsza metoda, wyniki przez nią zwracane są podobne do metody FAST, jako że korzysta ona z niej z pewnymi modyfikacjami, ale nieco gorsze

Dodatkowo warto zaznaczyć, że niektóre z metod potrafią również zwrócić dodatkowe informacje na temat znalezionych punktów, a niektóre nie. Do tych, które mogą zwracać dodatkowe informacje należą - SIFT, SURF, ORB. W rozwiązaniu nie skorzystano z tych informacji, ale na potrzeby innych zastosowań mogą być one bardzo przydatne.

### Wykrywanie twarzy na przykładowym filmie przy pomocy gotowego klasyfikatora kaskadowego.

In [ ]:
from ipywidgets import IntProgress
from IPython.display import display, clear_output

face_classifier = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

# przykładowe wideo pobrane za darmo ze strony
# https://www.vecteezy.com/video/5187030-portrait-of-young-beautiful-woman-with-smooth-healthy-skin-she-gently-touches-her-face-with-her-fingers-on-white-grey-background-skincare-concept
#####################################################################################################
cap = cv2.VideoCapture('vecteezy_portrait-of-young-beautiful-woman-with-smooth-healthy-skin__119.mov') # <- tu należy podać plik wideo/ścieżkę do niego
#####################################################################################################

vwidth  = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
vheight = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
max_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
vfps = cap.get(cv2.CAP_PROP_FPS)

frame_processing_progress = IntProgress(min=0, max=max_frames)
display(frame_processing_progress)


def detect_and_display(frame):
    frame_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    faces = face_classifier.detectMultiScale(frame_gray, 1.1, 4)
    for (x,y,w,h) in faces:
        center = (x + w//2, y + h//2)
        frame = cv2.ellipse(frame, center, (w//2, h//2), 0, 0, 360, (255, 0, 255), 4)
    return frame

fourcc = cv2.VideoWriter_fourcc(*'XVID')
out = cv2.VideoWriter('face_video.avi', fourcc, vfps, (vwidth, vheight))

while True:
    ret, frame = cap.read()
    if frame is None:
        break
    new_frame = detect_and_display(frame)

    out.write(new_frame)
    frame_processing_progress.value += 1
    frame_processing_progress.description = f'{frame_processing_progress.value} z {max_frames} ramek'

# usuwanie paska postępu
clear_output()

out.release()
cap.release()

Klasyfikator kaskadowy pozwala w prosty sposób wykryć pozycję twarzy w obrazie, ale nie jest wystarczająco szybki dla dużych obrazów. Czas przetwarzania przykładowego filmiku to ~2min (filmik trwa ~8s).

Przykładowy zrzut ekranu z wideo wynikowego dla szukania twarzy w obrazie.

![Przykładowy zrzut ekranu z działania znajdowania twarzy w przykładowym filmie wideo.](face_example.PNG)